# Berman Carbonate Database 
Required system packages and initialization

In [ ]:
import pandas as pd
import numpy as np
import sympy as sym
import os
import thermoengine
import dill

import importlib
sym.init_printing()



# Create coder module for pure phase using Berman Std State
* Head over to Berman-std-state-model Notebook to see coder expressions and model formulation

In [ ]:
%run core.ipynb
%run Berman-std-state-model.ipynb

In [ ]:
def Berman_params(Phase='Name', Formula='', 
                  H_TrPr=-1e6, S_TrPr=1e2, V_TrPr=3,
                  k0=1e2, k1=-10.0E2, k2=-5.0E5, k3=16.667E7,
                  v1=-1.0E-6, v2=0.0, v3=9.0E-6, v4=300.0E-10):
    
    param_dict = {
        'Phase': Phase, 'Formula': Formula,
        'T_r': 298.15, 'P_r': 1.0,
        'H_TrPr': H_TrPr, 'S_TrPr': S_TrPr, 'V_TrPr': V_TrPr,
        'k0': k0, 'k1': k1, 'k2': k2, 'k3': k3,
        'v1': v1, 'v2': v2, 'v3': v3, 'v4': v4,
        'l1': 0.0, 'l2': 0.0, 'k_lambda': 0.0,
        'T_lambda_Pr': 0.0, 'T_lambda_ref': 0.0, 'H_t': 0.0,
        'd0': 0.0, 'd1': 0.0, 'd2': 0.0, 'd3': 0.0, 'd4': 0.0, 'd5': 0.0,
        'T_D': 0.0, 'T_D_ref': 0.0}
    return param_dict

# Build up each phase from known properties
* taken from Berman 1988 for calcite, magnesite, dolomite
* other phases invented based on Robie (1995), Berman and Brown (1985), and NIST constraints

In [ ]:
phase_params = {}

In [ ]:
lookup_phase_info('calcite')

In [ ]:
phase_params['Cal'] = Berman_params(
    Phase='Calcite', Formula='Ca(1)C(1)O(3)',
    H_TrPr=-1206819.0, S_TrPr=91.725, V_TrPr=3.690,
    k0=178.19, k1=-16.577E2, k2=-4.827E5, k3=16.660E7,
    v1=-1.4E-6, v2=0.0, v3=8.907E-6, v4=227.402E-10)

In [ ]:
lookup_phase_info('magnesite')

In [ ]:
phase_params['Mgs'] = Berman_params(
    Phase='Magnesite', Formula='Mg(1)C(1)O(3)',
    H_TrPr=-1113636.0, S_TrPr=65.210, V_TrPr=2.803,
    k0=162.30, k1=-11.093E2, k2=-48.826E5, k3=87.466E7,
    v1=-0.890E-6, v2=2.212E-12, v3=18.436E-6, v4=415.968E-10)

In [ ]:
lookup_phase_info('aragonite')

In [ ]:
phase_params['Arg'] = Berman_params(
    Phase='Aragonite', Formula='Ca(1)C(1)O(3)',
    H_TrPr=-1206819.0+1100.0, S_TrPr=88.0, V_TrPr=3.415,
    k0=166.62, k1=-14.994E2, k2=0.0, k3=5.449E7,
    v1=-1.4E-6, v2=0.0, v3=8.907E-6, v4=227.402E-10)

In [ ]:
lookup_phase_info('siderite')

In [ ]:
phase_params['Sd'] = Berman_params(
    Phase='Siderite', Formula='Fe(1)C(1)O(3)',
    H_TrPr=-755900.0, S_TrPr=95.5, V_TrPr=2.938,
    k0=177.36, k1=-16.694E2, k2=-3.551E5, k3=15.078E7,
    v1=-0.890E-6, v2=2.212E-12, v3=18.436E-6, v4=415.968E-10)

In [ ]:
lookup_phase_info('dolomite')

In [ ]:
phase_params['Dol'] = Berman_params(
    Phase='Dolomite', Formula='Ca(1)Mg(1)C(2)O(6)',
    H_TrPr=-2324500.0+1100.0, S_TrPr=155.2, V_TrPr=6.434,
    k0=368.02, k1=-37.508E2, k2=0.0, k3=18.079E7,
    v1=-1.4E-6, v2=0.0, v3=8.907E-6, v4=227.402E-10)

# Build Databse

In [ ]:
mod_name = 'berman'

In [ ]:
modelDB, coder_mod = make_custom_database(mod_name, phase_params)

# Examine Phases in DB
* note Water and liq (melt) are always present according to MELTS and DEW models...

In [ ]:
modelDB.phase_info

# Save output phases for later

In [ ]:
output = {}

for phsnm in phase_params:
    iphs = modelDB.get_phase(phsnm)
    output[phsnm] = iphs
    

In [ ]:
output

In [ ]:
with open('carbon-phases.pkl','wb') as file:
    dill.dump(output, file)
    file.close()